# 🌾 Chana Quality Classifier - YOLOv8 Training

Train YOLOv8 model for real-time chana quality detection.

**Dataset**: 35,019 images, 9 classes

**Runtime**: Select GPU (Runtime → Change runtime type → T4 GPU)

## 1. Setup Environment

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Install dependencies
!pip install ultralytics -q
print("Ultralytics installed!")

## 2. Mount Google Drive & Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Unzip dataset from Google Drive
# Make sure you uploaded dataset.zip to your Drive root
!unzip -q /content/drive/MyDrive/dataset.zip -d /content/
print("Dataset extracted!")

In [ ]:
# Verify dataset
!ls /content/dataset/
!echo "Train images:" && ls /content/dataset/train/images | wc -l
!echo "Valid images:" && ls /content/dataset/valid/images | wc -l
!echo "Test images:" && ls /content/dataset/test/images | wc -l

## 3. Create Dataset Config

In [ ]:
%%writefile /content/data.yaml
path: /content/dataset
train: train/images
val: valid/images
test: test/images
nc: 9
names:
  0: Bad Black Channa
  1: Bad Kabuli Chana
  2: Good Black Channa
  3: Good Kabuli Chana
  4: foreign material
  5: Bad Soya
  6: Good Soya
  7: Bad Matar
  8: Good Matar

## 4. Train YOLOv8

In [ ]:
from ultralytics import YOLO

# Load pretrained model
model = YOLO('yolov8n.pt')  # nano for speed, use 'yolov8s.pt' for better accuracy

# Train
results = model.train(
    data='/content/data.yaml',
    
    # Training params
    epochs=100,
    batch=32,           # T4 can handle 32
    imgsz=640,
    
    # Optimizer
    optimizer='AdamW',
    lr0=0.01,
    
    # Augmentation (optimized for grains)
    degrees=180,        # Full rotation
    translate=0.1,
    scale=0.5,
    flipud=0.5,
    fliplr=0.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    mosaic=1.0,
    mixup=0.1,
    close_mosaic=10,
    
    # Hardware
    device=0,
    workers=4,
    amp=True,
    
    # Saving
    project='/content/drive/MyDrive/chana_models',
    name='yolov8n_v2',
    save=True,
    save_period=10,
    
    # Early stopping
    patience=20,
    
    # Validation
    val=True,
    plots=True,
)

## 5. Evaluate on Test Set

In [ ]:
# Load best model
best_model = YOLO('/content/drive/MyDrive/chana_models/yolov8n_v2/weights/best.pt')

# Evaluate on test set
results = best_model.val(
    data='/content/data.yaml',
    split='test',
    verbose=True
)

print(f"\nmAP50: {results.box.map50:.4f}")
print(f"mAP50-95: {results.box.map:.4f}")

## 6. Export Model

In [ ]:
# Export to ONNX
best_model.export(format='onnx', imgsz=640, simplify=True)

# Export to TensorRT (for NVIDIA deployment)
# best_model.export(format='engine', imgsz=640, half=True)

## 7. Test Inference

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import random
import os

# Get random test image
test_images = os.listdir('/content/dataset/test/images')
random_img = random.choice(test_images)
img_path = f'/content/dataset/test/images/{random_img}'

# Run inference
results = best_model(img_path)[0]

# Plot results
annotated = results.plot()
cv2_imshow(annotated)

## 8. Download Trained Model

In [ ]:
from google.colab import files

# Download best weights
files.download('/content/drive/MyDrive/chana_models/yolov8n_v2/weights/best.pt')